In [3]:
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import Ridge, Lasso
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import TransformerMixin, BaseEstimator
import pandas as pd
import numpy as np

class DataCleaner_v0(BaseEstimator, TransformerMixin):
    def __init__(self): # no *args and **kwargs
        super().__init__()
    
    def fit(self, X, y=None):
        return self  # nothing else to do
    
    def transform(self, X, y=None):   

        # Clean some punctutations
        data_temp=X.copy()
        data_temp = data_temp.str.replace('\n', ' \n ')
        # Remove ip address
        data_temp =    data_temp.str.replace(r'(([0-9]+\.){2,}[0-9]+)',' ')
        
        data_temp = data_temp.str.replace(r'([a-zA-Z]+)([/!?.])([a-zA-Z]+)',r'\1 \2 \3')
        # Replace repeating characters more than 3 times to length of 3
        data_temp= data_temp.str.replace(r'([*!?\'])\1\1{2,}',r'\1\1\1')
        # patterns with repeating characters 
        data_temp = data_temp.str.replace(r'([a-zA-Z])\1{2,}\b',r'\1\1')
        data_temp =data_temp.str.replace(r'([a-zA-Z])\1\1{2,}\B',r'\1\1\1')
        data_temp = data_temp.str.replace(r'[ ]{2,}',' ').str.strip()   
        # Add space around repeating characters
        data_temp =data_temp.str.replace(r'([*!?\']+)',r' \1 ')     
        return data_temp



model_path='./web/0.pkl'
with open(model_path,'rb') as fr:
    pipeline = pickle.load(fr)  

# while 1:
#     comment=input("comment:")   
#     score=pipeline.predict(pd.Series([comment]))
#     print(score)

In [4]:

df_rate_score=pd.read_csv("../input/jigsaw-toxic-severity-rating/comments_to_score.csv")
df_rate_sub=pd.read_csv("../input/jigsaw-toxic-severity-rating/sample_submission.csv")
df_rate_val=pd.read_csv("../input/jigsaw-toxic-severity-rating/validation_data.csv")

In [5]:
fld=0
n_folds=1

val_preds_arr1_tmp = np.zeros((df_rate_val.shape[0], n_folds))
val_preds_arr2_tmp = np.zeros((df_rate_val.shape[0], n_folds))


val_preds_arr1_tmp[:,fld] = pipeline.predict(df_rate_val['less_toxic'])
val_preds_arr2_tmp[:,fld] = pipeline.predict(df_rate_val['more_toxic'])

# print("\npredict test data ")
# test_preds_arr_tmp[:,fld] = pipeline.predict(clean_v0(df_rate_sub,'text')['text'])

p1 = val_preds_arr1_tmp.mean(axis=1)
p2 = val_preds_arr2_tmp.mean(axis=1)


val_score=np.round((p1 < p2).mean(),4)
print(f'Validation Accuracy is {val_score}')

<ipython-input-3-26457e59b9b6>:22: FutureWarning: The default value of regex will change from True to False in a future version.
  data_temp =    data_temp.str.replace(r'(([0-9]+\.){2,}[0-9]+)',' ')
<ipython-input-3-26457e59b9b6>:24: FutureWarning: The default value of regex will change from True to False in a future version.
  data_temp = data_temp.str.replace(r'([a-zA-Z]+)([/!?.])([a-zA-Z]+)',r'\1 \2 \3')
<ipython-input-3-26457e59b9b6>:26: FutureWarning: The default value of regex will change from True to False in a future version.
  data_temp= data_temp.str.replace(r'([*!?\'])\1\1{2,}',r'\1\1\1')
<ipython-input-3-26457e59b9b6>:28: FutureWarning: The default value of regex will change from True to False in a future version.
  data_temp = data_temp.str.replace(r'([a-zA-Z])\1{2,}\b',r'\1\1')
<ipython-input-3-26457e59b9b6>:29: FutureWarning: The default value of regex will change from True to False in a future version.
  data_temp =data_temp.str.replace(r'([a-zA-Z])\1\1{2,}\B',r'\1\1\1'

Validation Accuracy is 0.6913
